Creo l'ansatz $\ket{\text{oo-pUCCD}} = e^K e^{T^{\dagger}_{2}-T_2} \ket{\text{HF}}$ concatenando l'operatore $e^K$ con il circuito $\ket{\text{pUCCD}}=e^{T^{\dagger}_{2}-T_2} \ket{\text{HF}}$. 
L'obiettivo è definire, come cost function, il valore di aspettazione di $H$ con coefficienti modificati da $e^K$. 

## 0 - $\ket{\text{pUCCD}}$

In [90]:
''' ElectronicStructureProblem '''
import numpy as np
from scipy.linalg import expm
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers   import PySCFDriver
from typing import Tuple, Sequence, Union, List
from qiskit_nature.second_q.operators import FermionicOp, SparseLabelOp
from qiskit_nature.second_q.problems  import ElectronicBasis
from time import time

LiH = "Li 0 0 0; H 0 0 0.8" 

driver = PySCFDriver(
    atom=LiH,
    basis="sto3g", # 3 gaussiane per approssimare una funzione di Slater
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

mo_problem = driver.run()

''' Mapper '''
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

$$
   \ket{\text{pUCCD}} = e^{T^{\dagger}_{2}-T_2} \ket{\text{HF}}
$$

In [91]:
''' Ansatz '''
from qiskit_nature.second_q.circuit.library import HartreeFock, PUCCD

puccd = PUCCD(
    mo_problem.num_spatial_orbitals,
    mo_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        mo_problem.num_spatial_orbitals,
        mo_problem.num_particles,
        mapper,
    ),
)

''' Operators '''
problem_ops = mo_problem.second_q_ops()
main_op = mapper.map(problem_ops[0])
aux_ops = mapper.map(problem_ops[1])


Determino l'initial point dei primi (8) parametri del circuito $\ket{\text{pUCCD}}$ eseguendo una prima volta VQE. 
Dopodiché le alternative sono due:
- fissare i primi parametri e ottimizzare solo i (15) $k$ orbitali
- ottimizzare ogni volta tutti i parametri

in questo programma si adotta la seconda strategia.

In [92]:
from qiskit_algorithms import VQE, VQEResult
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator # Estimator deprecated 

estimator = Estimator()
ini_solver = VQE(estimator, puccd, SLSQP())
ini_result = ini_solver.compute_minimum_eigenvalue(operator=main_op, aux_operators=aux_ops)
ini_point = ini_result.optimal_point

/var/folders/88/gfgbzsfs2m759nlsp4wspv400000gp/T/ipykernel_30437/3687483880.py:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [93]:
# energia puccd: -7.316485492334664
interpreted = mo_problem.interpret(ini_result)
energy = interpreted.groundenergy + interpreted.nuclear_repulsion_energy
print(energy)

-7.63118682891578


# 1 - $\ket{\text{oo-pUCCD}}$

### 1.1 - Excitations

$$
    K = \sum_{p<q}\sum_{\sigma} k_{pq} 
    \left(
        a^{\dagger}_{p\sigma}a^{}_{q\sigma} - a^{\dagger}_{q\sigma}a^{}_{p\sigma}
    \right)
$$

In [94]:
def create_orbital_rotation_list(ansatz) -> list:
    """ Creates a list of indices of matrix kappa that denote the pairs of orbitals that
    will be rotated. For instance, a list of pairs of orbital such as [[0,1], [0,2]]. """
    
    half_as = int(ansatz.num_qubits / 2)
    
    orbital_rotations = []
    
    # TODO: 
    '''
    # list is built according to frozen orbitals
    if self._frozen_list:
        for i in range(half_as):
            if i in self._frozen_list:
                continue
            for j in range(half_as):
                if j in self._frozen_list:
                    continue
                if i < j:
                    self._orbital_rotations.append([i, j])
    else:
        for i in range(half_as):
            for j in range(half_as):
                if i < j:
                    self._orbital_rotations.append([i, j])
    '''
    
    for i in range(half_as):
            for j in range(half_as):
                if i < j:
                    orbital_rotations.append([i, j])
                    
    return orbital_rotations

        

In [95]:
from qiskit.circuit.library import EvolvedOperatorAnsatz

rotations = create_orbital_rotation_list(ansatz=puccd)
print (rotations)

excitations = [([i], [j]) for i, j in rotations]
print(excitations)

[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
[([0], [1]), ([0], [2]), ([0], [3]), ([0], [4]), ([0], [5]), ([1], [2]), ([1], [3]), ([1], [4]), ([1], [5]), ([2], [3]), ([2], [4]), ([2], [5]), ([3], [4]), ([3], [5]), ([4], [5])]


### 1.2 - Excitation operator $K$

La funzione **build_fermionic_excitation_ops** è quella definita in qiskit.circuit.ansatzes.UCC

In [96]:
def build_fermionic_excitation_ops(excitations: Sequence) -> list[FermionicOp]:
    """Builds all possible excitation operators with the given number of excitations for the
    specified number of particles distributed in the number of orbitals.

    Args:
        excitations: the list of excitations.

    Returns:
        The list of excitation operators in the second quantized formalism.
    """
    num_spin_orbitals = 2 * mo_problem.num_spatial_orbitals
    operators = []

    for exc in excitations:
        label = []
        for occ in exc[0]:
            label.append(f"+_{occ}")
        for unocc in exc[1]:
            label.append(f"-_{unocc}")
        op = FermionicOp({" ".join(label): 1}, num_spin_orbitals=num_spin_orbitals)
        op_adj = op.adjoint()
        # we need to account for an additional imaginary phase in the exponent accumulated from
        # the first-order trotterization routine implemented in Qiskit
        op_minus = 1j * (op - op_adj)
        operators.append(op_minus)

    return operators

In [97]:
ops = build_fermionic_excitation_ops(excitations=excitations)
print (ops)

qubit_ops = mapper.map(ops)

[FermionicOp({'+_0 -_1': 1j, '+_1 -_0': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_0 -_2': 1j, '+_2 -_0': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_0 -_3': 1j, '+_3 -_0': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_0 -_4': 1j, '+_4 -_0': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_0 -_5': 1j, '+_5 -_0': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_1 -_2': 1j, '+_2 -_1': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_1 -_3': 1j, '+_3 -_1': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_1 -_4': 1j, '+_4 -_1': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_1 -_5': 1j, '+_5 -_1': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_2 -_3': 1j, '+_3 -_2': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_2 -_4': 1j, '+_4 -_2': np.complex128(-0-1j)}, num_spin_orbitals=12, ), FermionicOp({'+_2 -_5': 1j, '+_5 -_2': np.

### 1.3 - oo-pUCCD circuit

$$
   \ket{\text{oo-pUCCD}} = e^K e^{T^{\dagger}_{2}-T_2} \ket{\text{HF}}
$$

In [98]:
# concateno e^K a |pUCCD>
oo_puccd = EvolvedOperatorAnsatz(
    operators=qubit_ops,
    initial_state=puccd,
    parameter_prefix='k'
)

In [99]:
print('qubits: ', oo_puccd.num_qubits)
print('params: ', oo_puccd.num_parameters)
print('bounds: ', oo_puccd.parameter_bounds)

qubits:  12
params:  23
bounds:  None


## 2 - Orbital Rotations

### 2.1 - Orbital rotation matrix

$$
    e^K
$$

In [100]:
rotation_parameters = oo_puccd.parameters[puccd.num_parameters:]
dim_kappa_matrix = mo_problem.num_spatial_orbitals

def orbital_rotation_matrix(parameters: np.ndarray, rotations: list) -> Tuple[np.ndarray, np.ndarray]:
    """ Creates 2 matrices K_alpha, K_beta that rotate the orbitals through MO coefficient
    C_alpha = C_RHF * U_alpha where U = e^(K_alpha), similarly for beta orbitals. """

    k_matrix_alpha = np.zeros((dim_kappa_matrix, dim_kappa_matrix))
    k_matrix_beta  = np.zeros((dim_kappa_matrix, dim_kappa_matrix))
    
    for i, exc in enumerate(rotations):
        k_matrix_alpha[exc[0]][exc[1]] =  parameters[i]
        k_matrix_alpha[exc[1]][exc[0]] = -parameters[i]
        k_matrix_beta[exc[0]][exc[1]]  =  parameters[i]
        k_matrix_beta[exc[1]][exc[0]]  = -parameters[i]

    matrix_a = expm(k_matrix_alpha)
    matrix_b = expm(k_matrix_beta)
    
    return matrix_a, matrix_b

### 2.2 - Integrals

Prendo gli integrali da PySCF

In [101]:
from pyscf import gto

# Molecola idruro di litio
LiH = "Li .0 .0 .0; H .0 .0 0.600"
mol = gto.M(atom=LiH, basis='sto3g')
two_electron_integrals = mol.intor('int2e') # , aosym='s8') opzioni di simmetria
one_electron_integrals = mol.intor("int1e_kin") + mol.intor("int1e_nuc")

Modifico gli integrali con $e^K$: 

$$
    \tilde{h}_{pq} = \sum_{uv} C_{up} h_{uv} C_{vq}\quad
    \land\quad
	(\tilde{p}\tilde{q}|\tilde{r}\tilde{s})=\sum_{uvxy} C_{up}C_{vq}(pq|rs)C_{xr}C_{ys}
$$

con $C_{pq} = \left\{e^K\right\}_{pq}$

In [102]:
from qiskit_nature.second_q.operators import ElectronicIntegrals

def rotate_orbitals(mo_problem, matrix_a, matrix_b): 
    """Doctstring"""
    
    # Matrice C (6x6) e integrali a un corpo (6x6)
    C = matrix_a  
    h = one_electron_integrals
    
    N = len(C)

    # Inizializzo matrice per gli integrali trasformati
    h_transformed = np.zeros((N, N))

    # Trasformazione degli integrali a un corpo
    for p in range(N):
        for q in range(N):
            sum_value = 0
            for u in range(N):
                for v in range(N):
                    sum_value += C[u, p] * h[u, v] * C[v, q]
            h_transformed[p, q] = sum_value
            
    # Trasformazione degli integrali a due corpi (si potrebbe evitare di ri-ciclare su p e q)
    eri_transformed = np.zeros((N, N, N, N))

    for p in range(N):
        for q in range(N):
            for r in range(N):
                for s in range(N):
                    sum_value = 0
                    for u in range(N):
                        for v in range(N):
                            for x in range(N):
                                for y in range(N):
                                    sum_value += C[u, p] * C[v, q] * two_electron_integrals[u, v, x, y] * C[x, r] * C[y, s]
                    eri_transformed[p, q, r, s] = sum_value
    
    # definisco un oggetto ElectronicIntegrals per modificare quello contenuto in mo_problem
    e_int = ElectronicIntegrals.from_raw_integrals(h_transformed, eri_transformed)

    mo_problem.hamiltonian.ElectronicIntegrals = e_int
    
    # extract rotated operator
    rotated_operator = mo_problem.hamiltonian.second_q_op()
    rotated_operator = mapper.map(rotated_operator)
    
    return rotated_operator

# 3 - VQE

VQE sul circuito restituisce 23 parametri ottimizzati. I primi 8 sono i parametri $t$ di **puccd**, gli altri 15 $k$ sono usati per modificare i coefficienti orbitali. Ad ogni fase intermedia dell'ottimizzazione gli operatori vengono ricalcolati.

Creo istanza vqe e estraggo il primo optimal point per i parametri puccd

In [103]:
''' VQE instantiation '''
#vqe_solver = VQE(estimator, oo_puccd, SLSQP(), gradient=gradient)
vqe_solver = VQE(estimator, oo_puccd, SLSQP())
# inizializzo i parametri, devo solo vedere se funziona
vqe_solver.initial_point = [0.0] * oo_puccd.num_parameters
vqe_solver.initial_point[:puccd.num_parameters] = ini_point

In [104]:
interpreted = mo_problem.interpret(ini_result)

In [105]:
interpreted.groundenergy+interpreted.nuclear_repulsion_energy
# VQE su pUCCD: -7.316485491073372

np.float64(-7.63118682891578)

L'idea del codice è minimizzare **energy_evaluation_oo**, che ruota l'operatore e calcola il valore di aspettazione.

In [106]:
def energy_evaluation_oo(
    solver, 
    rotations,
    num_parameters_puccd: int,
    parameters: np.ndarray
) -> Union[float, List[float]]:
    """Doctstring"""
    
    # splicing
    puccd_parameter_values    = parameters[:num_parameters_puccd ] 
    rotation_parameter_values = parameters[ num_parameters_puccd:] 

    # CALCULATE COEFFICIENTS OF ROTATION MATRIX HERE:
    matrix_a, matrix_b = orbital_rotation_matrix(rotation_parameter_values, rotations)
    
    #print('pUCCD parameters:\n', puccd_parameter_values)
    #print('Rotation parameters:\n', rotation_parameter_values)
    #print("Nature matrix a:\n", matrix_a)
    #print("Nature matrix b:\n", matrix_b)
    
    # ROTATE AND RECOMPUTE OPERATOR HERE:
    rotated_operator = rotate_orbitals(mo_problem, matrix_a, matrix_b)
        
    try:
        job = solver.estimator.run(solver.ansatz, rotated_operator, parameters)
        estimator_result = job.result()
    except Exception as exc:
        raise KeyError("The primitive job to evaluate the energy failed!") from exc

    values = estimator_result.values
    
    energy = values[0] if len(values) == 1 else values

    # the rest of the energy evaluation code only involves the ansatz parameters

    return energy

Minimizzo

In [107]:
from functools import partial 

energy_evaluation = partial(energy_evaluation_oo, vqe_solver, rotations, puccd.num_parameters)

# setto bounds default
bounds = [[-2*np.pi,2*np.pi] for _ in range(oo_puccd.num_parameters)]
# setto un initial point più vicino al risultato finale
vqe_solver.initial_point = \
[-3.10790974e+00, -3.88783726e-04, -6.28318531e+00, -6.28318531e+00,
 -3.14240338e+00,  4.14092583e-02,  3.14159779e+00, -1.14763784e-04,
  1.37528292e-02,  1.74358163e-03,  2.29391354e-03, -3.88335215e-01,
 -6.39069234e-05,  2.01296231e-03,  2.72899940e-03, -2.70109444e-03,
  1.09881611e-03, -1.10917682e-03,  1.00909373e-02,  6.82633545e-03,
  7.23208646e-02, -7.25136257e-02,  1.01475338e-01]


start_time = time()

# minimization
opt_result = vqe_solver.optimizer.minimize(
    fun=energy_evaluation, x0=vqe_solver.initial_point, bounds=bounds
)
    
eval_time = time() - start_time


Per manipolare il risultato serve estrarre i dati dal raw_result

In [108]:
result = VQEResult()
result.optimal_point = opt_result.x
result.optimal_parameters = dict(zip(vqe_solver.ansatz.parameters, opt_result.x))
result.optimal_value = opt_result.fun
result.cost_function_evals = opt_result.nfev
result.optimizer_time = eval_time
result.eigenvalue = opt_result.fun + 0j

In [109]:
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 272,
    'eigenvalue': np.complex128(-9.616853072690386+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(k[5]): np.float64(0.026062103156729372),
                              ParameterVectorElement(k[3]): np.float64(-6.283182545808763),
                              ParameterVectorElement(k[4]): np.float64(-3.1411897068845267),
                              ParameterVectorElement(t[0]): np.float64(-0.003749305887797703),
                              ParameterVectorElement(t[5]): np.float64(0.04662442633747548),
                              ParameterVectorElement(t[1]): np.float64(0.0013898785195582746),
                              ParameterVectorElement(t[3]): np.float64(0.0026535866508661868),
                              ParameterVectorElement(t[7]): np.float64(0.1098649230263283),
                              ParameterVectorElement(t[6]): np.float64(-0.04667921563599227),
   

In [110]:
oo_interpreted = mo_problem.interpret(result)
energy = oo_interpreted.groundenergy + oo_interpreted.nuclear_repulsion_energy
print(energy)

-7.632438531740386


Energie 
- puccd -7.63118682891578
- oo_puccd -7.632438531740386